In [3]:
"""
============================================================
Mouse Behavior Video Data Collection Interface
============================================================

This script implements a Gradio-based web interface for collecting 
mouse behavior video data, including experimental details, video 
uploads, and region-of-interest (arena) annotations.

Features:
- Collects experimental metadata (date, provider info, lighting, etc.)
- Allows users to upload video files and extracts the middle frame
- Provides an interactive arena annotation tool for labeling video frames
- Saves all collected data as structured JSON files

Author: [Meng-Xuan Liu]
Date Created: [2025-03-11]
Last Modified: [2025-03-11]
============================================================
"""

import gradio as gr
import os
import shutil
import json
from datetime import datetime
from utils import video_utils  # 假設此模組已正確實現影片檢查與中間幀擷取

# ------------------------------
# 實驗資訊組合函式
# ------------------------------
def submit_experiment_info(exp_date, provider_unit, provider_name, lighting_conditions, camera_equipment, notes):
    return {
        "Experiment Date": exp_date,
        "Provider's Unit": provider_unit,
        "Provider's Name": provider_name,
        "Lighting Conditions": lighting_conditions,
        "Camera Equipment": camera_equipment,
        "Additional Notes": notes
    }

# ------------------------------
# 影片上傳與中間幀擷取
# ------------------------------
def handle_video_upload(file):
    if file is None:
        return None, "請上傳影片檔案。", ""

    # 設定存放資料夾
    raw_dir = "data/uploads/raw"
    metadata_dir = "data/uploads/metadata"
    os.makedirs(raw_dir, exist_ok=True)
    os.makedirs(metadata_dir, exist_ok=True)

    # 產生影片名稱
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    new_video_filename = f"video_{timestamp}.mp4"
    new_video_path = os.path.join(raw_dir, new_video_filename)

    # 確保檔案來源正確
    source_path = getattr(file, 'path', file.name)
    
    # 改用 move，避免多餘的 I/O 操作
    shutil.move(source_path, new_video_path)

    # 擷取影片中間幀 (僅用於前端 arena 標註，不存入最終 JSON)
    output_dir = "static/frames"
    os.makedirs(output_dir, exist_ok=True)
    output_image = f"{output_dir}/frame_{timestamp}.png"

    valid, message = video_utils.check_video_file(new_video_path)
    if not valid:
        return None, message, new_video_path

    success = video_utils.extract_middle_frame(new_video_path, output_image)
    if not success:
        return None, "Failed to extract middle frame from video.", new_video_path

    return output_image, "Video uploaded successfully", new_video_path

# ------------------------------
# 最終提交函式
# ------------------------------
def validate_submission(exp_date, provider_unit, provider_name, lighting_conditions, camera_equipment, notes, video_file, arena_data):
    """ 檢查是否所有必填欄位皆已填寫，影片上傳、arena 標註是否完整 """

    # 1️⃣ 檢查實驗資訊 (不可空白)
    required_fields = {
        "Experiment Date": exp_date.strip(),
        "Provider's Lab": provider_unit.strip(),
        "Provider's Name": provider_name.strip(),
        "Camera Equipment": camera_equipment.strip()
    }

    for field, value in required_fields.items():
        if not value:
            return f"⚠️ Error: '{field}' is required and cannot be empty."

    # 2️⃣ 檢查影片是否已上傳
    if not video_file or not os.path.exists(video_file):
        return "⚠️ Error: A video file must be uploaded before submission."

    # 3️⃣ 檢查至少有一個 arena
    try:
        arena_info = json.loads(arena_data) if arena_data else []
    except json.JSONDecodeError:
        return "⚠️ Error: Invalid arena data format."

    if len(arena_info) == 0:
        return "⚠️ Error: At least one arena must be defined before submission."

    return None  # 所有檢查通過


def final_submit_data(exp_date, provider_unit, provider_name, lighting_conditions, camera_equipment, notes, video_file, arena_data):
    """ 確保所有必要資料皆填寫後，才將資料儲存為 JSON """
    
    # 進行表單完整性檢查
    validation_error = validate_submission(exp_date, provider_unit, provider_name, lighting_conditions, camera_equipment, notes, video_file, arena_data)
    if validation_error:
        return validation_error  # 若有錯誤則回傳錯誤訊息，不儲存

    # 建立實驗資訊 JSON
    experiment_info = submit_experiment_info(exp_date, provider_unit, provider_name, lighting_conditions, camera_equipment, notes)
    video_info = {
        "video_file": video_file,
        "uploaded_at": datetime.now().strftime("%Y%m%d_%H%M%S")
    }

    arena_info = json.loads(arena_data)

    final_data = {
        "experiment_info": experiment_info,
        "video_info": video_info,
        "arena": arena_info
    }

    # 儲存 JSON
    metadata_dir = "data/uploads/metadata"
    os.makedirs(metadata_dir, exist_ok=True)
    final_json_path = os.path.join(metadata_dir, f"video_{video_info['uploaded_at']}.json")
    
    with open(final_json_path, "w", encoding="utf-8") as f:
        json.dump(final_data, f, ensure_ascii=False, indent=4)

    return "✅ All data submitted successfully!"



html_template = '''
<div id="arena-container" border:0px solid #ccc;"></div>
<input type="hidden" id="arena_data">
'''

# ------------------------------
# 讀取 JavaScript 檔案 (arena 處理) & CSS 檔案
# ------------------------------
with open("utils/arenas_darwer_js.txt", "r", encoding="utf-8") as f:
    head_html = f.read()

with open("utils/css.txt", "r", encoding="utf-8") as f:
    CSS_setting = f.read()

# ------------------------------
# Gradio 介面
# ------------------------------
with gr.Blocks(head = head_html, css = CSS_setting) as demo:
    
    gr.Markdown("# Mouse Behavior Video Data Collection")
    
    # --- Section 1: 實驗資訊 (Experimental Information) ---
    with gr.Column():
        gr.Markdown("## 🧪 1. Experimental Information")

        # 讓短欄位 (日期、單位、姓名) 並排顯示
        with gr.Row():
            exp_date = gr.Textbox(label="📅 Experiment Date", 
                                value=datetime.now().strftime("%Y-%m-%d"), 
                                placeholder="YYYY-MM-DD", 
                                interactive=True)
            
            provider_unit = gr.Textbox(label="🏢 Provider's Lab", 
                                    placeholder="e.g., Hsu Lab", 
                                    interactive=True)
            
            provider_name = gr.Textbox(label="🧑‍🔬 Provider's Name", 
                                    placeholder="e.g., Hideo Kojima", 
                                    interactive=True)

        # 其餘較長的欄位維持直列顯示

        camera_equipment = gr.Textbox(label="📷 Camera Equipment", 
                            placeholder="e.g., Sony A7 III, 50mm f/1.8 lens", 
                            interactive=True)
        
        lighting_conditions = gr.Textbox(label="💡 Lighting Conditions", 
                                        placeholder="Optional: e.g., Ceiling fluorescent lamp, brightness measured at bottom: 1000 lux", 
                                        interactive=True)
        
        notes = gr.Textbox(label="📝 Additional Notes", 
                        placeholder="Optional: any extra information about the experiment",
                        interactive=True, 
                        lines=3)


    # --- Section 2: 影片上傳 (Video Upload) ---
    with gr.Column():
        gr.Markdown("## 🎞️ 2. Video Upload")

        # 讓 "Upload Video File" & "Upload Status" 並排顯示
        with gr.Row():
            with gr.Column(scale=2):
                video_input = gr.File(label="📂 Upload Video File (Supported formats: mp4, avi, mts, mov)", 
                                    interactive=True)

            with gr.Column(scale=1):
                status_output = gr.Textbox(label="📢 Upload Status", 
                                        interactive=False, 
                                        placeholder="Waiting for upload...")

        # 顯示擷取的中間幀 (加入邊框以突顯)
        image_output = gr.Image(label="🖼️ Middle Frame Preview", 
                                type='filepath', 
                                interactive=False,
                                container=True, visible=False)  # 讓圖片有額外的邊框

        # 隱藏欄位存放影片檔案路徑 (用於後續處理)
        video_path_hidden = gr.Textbox(label="Video File Path", 
                                    visible=False)

        # 影片上傳觸發處理函式
        video_input.upload(
            handle_video_upload,
            inputs=video_input,
            outputs=[image_output, status_output, video_path_hidden]
        )


        # --- Section 3: arena 標註 (畫布 + 按鈕) (arena Annotation) ---
        gr.Markdown("## 3. arena Annotation 🎯")

        # 使用 Row 讓畫布與按鈕區域並排
        with gr.Row():
            # 右側：畫布 (Canvas) 設為較大區域
            with gr.Column(scale=1):  # 空白區域
                pass

            with gr.Column(scale=4):  # 主要畫布
                arena_canvas = gr.HTML(value=html_template, label="arena Annotation Canvas")

            # 左側：按鈕區域 (Buttons)
            with gr.Column(scale=1):  
                add_arena_btn = gr.Button("➕ Add arena")
                remove_arena_btn = gr.Button("🗑 Remove arena")
                select_canvas_btn = gr.Button("🖼 Whole canvas")

        # 自動初始化畫布（當影片上傳並擷取中間幀時）
        image_output.change(
            fn=None,
            inputs=[image_output],
            outputs=None,
            js="(imageURL) => initKonva(imageURL)"
        )

        # 按鈕功能設定
        add_arena_btn.click(
            fn=None,
            inputs=None,
            outputs=None,
            js="() => { createNewarena(); return ''; }"
        )
        remove_arena_btn.click(
            fn=None,
            inputs=None,
            outputs=None,
            js="() => { removeSelectedarena(); return ''; }"
        )
        select_canvas_btn.click(
            fn=None,
            inputs=None,
            outputs=None,
            js="() => { selectWholeCanvasarena(); return ''; }"
        )



    # --- Section 4: 最終提交 (Final Submission) ---
    with gr.Column():
        gr.Markdown("## 4. Submission📤")
        arena_data_hidden = gr.Textbox(label="arena Data", visible=False, value="")
        final_submit_btn = gr.Button("🚀 Submit All Data")
        final_status = gr.Textbox(label="Submission Status", interactive=False)
        
        final_submit_btn.click(
            final_submit_data,
            inputs=[exp_date, provider_unit, provider_name, lighting_conditions, camera_equipment, notes, video_path_hidden, arena_data_hidden],
            outputs=final_status,
            js="""
            (exp_date, provider_unit, provider_name, lighting_conditions, camera_equipment, notes, video_file, arena_data_hidden) => {
                updatearenaList();  // 確保更新 arena 資料
                let arenaData = document.getElementById("arena_data").value;  // 讀取已更新的 arena JSON
                return [exp_date, provider_unit, provider_name, lighting_conditions, camera_equipment, notes, video_file, arenaData];
            }
            """
        )



demo.launch(share=True)  # 啟動 Gradio 介面

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://5f3be2c1cc09620fe9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
